In [1]:
import numpy as np 
import pandas as pd
import os
import io
import glob
import re
import sys
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import animation, rc
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
import plotly.express as px
import plotly.graph_objects as go
import folium
from folium import plugins
import branca
import json

import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
files = glob.glob('dataset/*.csv')
data = []
for i in files :
    i = re.sub("dataset/", "", i)
    data.append(re.sub(".csv","",i))
for i in range(len(files)) :
    globals()[data[i]] = pd.read_csv(files[i])
data

['SeoulFloating',
 'TimeAge',
 'SearchTrend',
 'TimeProvince',
 'Weather',
 'PatientRoute',
 'PatientInfo',
 'Region',
 'TimeGender',
 'Policy',
 'Case',
 'Time']

In [6]:
timedat = Time.set_index('date').drop('time',axis=1)

In [13]:
dailydata = timedat.diff()

In [15]:
dailydata.iloc[0,:] = [0,0,0,0,0]

In [19]:
ratio = dailydata.confirmed/dailydata.test
ratio[ratio.isnull()] = 0
dailydata['ratio'] = ratio
total = Time[Time.date == '2020-04-20'].confirmed
totaltest = Time[Time.date == '2020-04-20'].test
dailydata['confirmed'] = dailydata.confirmed/total.tolist()
dailydata['test'] = dailydata.test/totaltest.tolist()

In [53]:
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=dailydata.index, y=dailydata.ratio, name='Ratio',
                         line=dict(color='black', width=3)))
fig1.add_trace(go.Scatter(x=dailydata.index, y=dailydata.confirmed, name = 'Confirmed',
                         line=dict(color='royalblue', width=2)))
fig1.add_trace(go.Scatter(x=dailydata.index, y=dailydata.test, name='Test',
                         line=dict(color='dimgrey', width=2)))
fig1.show()

ratio = 확진자수/검사수

confirmed, test : 값을 총합으로 나누어 standardize

ratio 가 크다 : 검사수는 적었으나, 검사한 사람들 중 확진자의 비율이 많았다. ; 검사대상이 된 사람들이 대거 확진되었다.

ratio 가 작다 : 검사수는 많았으나, 검사한 사람들 중 확진자의 비율이 작았다. ; 검사대상이 된 사람들이 감염 요인이 있었음에도 불구하고 확진되지 않았다.

---
ratio 는 3월 1일까지 상승경향을 보이다가 이후 지속적인 하락. 

3월 1일 이전의 ratio 상승은 확진자 상승 영향으로 인한 것이지만 3월 29일 ratio의 jump는 test의 감소로 인한 것임.

전체적인 경향으로 보면 test 값도 서서히 하락하고는 있다. 아 뭔가 보기 힘든거같은데

In [56]:
standardized = timedat.loc[:,:].div(timedat.iloc[-1])

In [58]:
fig2 = standardized.iplot(asFigure=True,kind='scatter')
fig2.show()

아 걍 애초부터 이렇게 볼걸

누적 검사수는 기울기가 거의 1이다

확진자수는 시그모이드 모양

즉 검사수는 일정한데 확진자수는 줄어들었단 소리

In [66]:
rsv = pd.read_csv('custom_data/rsv.csv',header=None)
rsv = rsv.set_index([0,1])

In [68]:
rsv = rsv.drop(3,axis=1)
rsv.columns = ['rsv']

In [69]:
rsv

rsv
0    1        
2015 1   577.0
     2   519.0
     3   387.0
     4   281.0
     5   299.0
...        ...
2020 13   17.0
     14   17.0
     15   19.0
     16    8.0
     17    NaN

[278 rows x 1 columns]

In [70]:
fig4 = rsv.iplot(asFigure=True, kind='scatter')
fig4.show()

주기가 있다. standardize 해서 corona의 예측에 사용할 수 있을까?

In [77]:
timedat = Time.set_index('date').drop('time',axis=1)
dailyconf = timedat.confirmed.diff()
dailyconf[0] = 1
dailyconf = pd.DataFrame(dailyconf)
dailyconf.index = pd.to_datetime(dailyconf.index)
dailyconf['week'] = dailyconf.index.week
dailyconf['year'] = pd.DatetimeIndex(Time.date).year

In [80]:
std_corona = weekconf/max(weekconf.confirmed)

In [81]:
comb = pd.concat([std_rsv,std_corona],axis=1)
comb.columns = ['rsv','corona']

In [83]:
fig5 = comb.iplot(asFigure=True,kind='scatter')
fig5.show()

In [87]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [136]:
decomp = seasonal_decompose(rsv[rsv.rsv.notnull()],model='additive',period=52)

In [137]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=4, cols=1)

fig.add_trace(go.Scatter(y=decomp.observed),
              row=1, col=1)

fig.add_trace(go.Scatter(y=decomp.resid),
              row=2, col=1)

fig.add_trace(go.Scatter(y=decomp.trend),
              row=3, col=1)

fig.add_trace(go.Scatter(y=decomp.seasonal),
              row=4, col=1)

fig.show()

In [150]:
temp = decomp.seasonal.unique()

In [156]:
temp  = np.roll(temp, shift= -26)

In [157]:
temp = pd.DataFrame(temp)

In [158]:
fig = temp.iplot(asFigure=True,kind='scatter')
fig.show()

In [160]:
timedat = Time.set_index('date').drop('time',axis=1)
dailydata = timedat.diff()
dailydata.iloc[0,:] = [0,0,0,0,0]

In [165]:
len(temp)

52

In [163]:
fig = dailydata.confirmed.iplot(asFigure=True,kind='scatter')
fig.show()

In [164]:
len(dailydata)

92

## 막힌 부분 : rsv의 seasonality 를 corona에 어떻게 적용시키는거지??

1. 한 주기의 length 가 서로 다름
2. 